Library

In [1]:
import cv2 as cv
import numpy as np
import time
from matplotlib import pyplot as plt
from math import radians, sin, cos, pi, sqrt

Build Pyramid Image

In [2]:
def buildPyramid(image, maxleval):
    imgpyr = [image]
    aux = image
    for i in range(0, maxleval):
        aux = cv.pyrDown(aux)
        imgpyr.append(aux)

    imgpyr.reverse()
    return imgpyr

Rotation

In [30]:
def rotation(image, angleInDegrees):
    h, w = image.shape[:2]
    img_c = (w/2, h/2)

    rot = cv.getRotationMatrix2D(img_c, angleInDegrees, 1)
    rad = radians(angleInDegrees)
    si = sin(rad)
    co = cos(rad)
    b_w = int((h * abs(si)) + (w * abs(co)))
    b_h = int((h * abs(co)) + (w * abs(si)))

    rot[0, 2] += ((b_w / 2) - img_c[0])
    rot[1, 2] += ((b_h / 2) - img_c[1])

    outImg = cv.warpAffine(image, rot, (b_w, b_h), flags=cv.INTER_LINEAR)
    return outImg

Recangle

In [31]:
def recangleX(refimg, W, H, Wt, Ht, goc, pt):
    anglerad = radians(goc)
    point1 = (-W // 2, -H // 2)
    point2 = (W // 2, -H // 2)
    point3 = (W // 2, H // 2)
    point4 = (-W // 2, H // 2)
    
    point1 = (int(point1[0] * cos(2 * pi - anglerad) - point1[1] * sin(2 * pi - anglerad)),
              int(point1[0] * sin(2 * pi - anglerad) + point1[1] * cos(2 * pi - anglerad)))
    point2 = (int(point2[0] * cos(2 * pi - anglerad) - point2[1] * sin(2 * pi - anglerad)),
              int(point2[0] * sin(2 * pi - anglerad) + point2[1] * cos(2 * pi - anglerad)))
    point3 = (int(point3[0] * cos(2 * pi - anglerad) - point3[1] * sin(2 * pi - anglerad)),
              int(point3[0] * sin(2 * pi - anglerad) + point3[1] * cos(2 * pi - anglerad)))
    point4 = (int(point4[0] * cos(2 * pi - anglerad) - point4[1] * sin(2 * pi - anglerad)),
              int(point4[0] * sin(2 * pi - anglerad) + point4[1] * cos(2 * pi - anglerad)))

    point1 = (point1[0] + pt[0] + Wt // 2, point1[1] + pt[1] + Ht // 2)
    point2 = (point2[0] + pt[0] + Wt // 2, point2[1] + pt[1] + Ht // 2)
    point3 = (point3[0] + pt[0] + Wt // 2, point3[1] + pt[1] + Ht // 2)
    point4 = (point4[0] + pt[0] + Wt // 2, point4[1] + pt[1] + Ht // 2)
    cv.line(refimg, point1, point2, (0, 0, 0), 2)
    cv.line(refimg, point2, point3, (0, 0, 0), 2)
    cv.line(refimg, point3, point4, (0, 0, 0), 2)
    cv.line(refimg, point4, point1, (0, 0, 0), 2)

Pyramid rotated matching template

In [36]:
def fastTemplateMatchPyramid(src_refimg, src_tplimg, maxleval):
    ## Build image pyramid
    refimgs = buildPyramid(src_refimg, maxleval)
    tplimgs = buildPyramid(src_tplimg, maxleval)

    ## Do template match
    top_lefts = []
    step = 0
    for idx in range(0, maxleval + 1):
        refimg = refimgs[idx]
        tplimg = tplimgs[idx]
        W, H = refimg.shape[::-1]
        Wt, Ht = tplimg.shape[::-1]
        print('Level', idx)
        step_save = step
        step = int(round(sqrt(2)/((sqrt(Ht**2+Wt**2))*pi)*360, 1)*10)
        masktpl = np.uint8(np.ones((Ht, Wt)))
        if idx == 0:
            ans = []
            print(Wt, Ht)
            print(step)
            for angles in range(0, 3600, step): 
                angle = angles/10
                tplimg_new = rotation(tplimg, angle)
                anglerad = radians(angle)
                masktpl_new = rotation(masktpl, angle)
                result = cv.matchTemplate(refimg, tplimg_new, cv.TM_CCORR_NORMED, mask = masktpl_new)
                minval, maxval, minloc, maxloc = cv.minMaxLoc(result)
                if maxval > 0.9:
                    ans.append([result, angle, maxloc])
            
        else:
            ans_new = []
            for res, goc, topleft in ans:
                loc = None
                d = goc
                res = None
                k = step_save
                y = topleft[1]*2
                x = topleft[0]*2
                print(Wt, Ht)
                print(step)
                print(d, topleft)
                print(int(d*10-k), int(d*10+k+1))
                for angles in range(int((d - k)*10), int((d + k + 1)*10), step):
                    angle = angles/10
                    tplimg_new = rotation(tplimg, angle)
                    w, h = tplimg_new.shape[::-1]
                    masktpl_new = rotation(masktpl, angle)
                    refimg_new = refimg[y-2:y+h+1, x-2:x+w+1]
                    result = cv.matchTemplate(refimg_new, tplimg_new, cv.TM_CCORR_NORMED, mask = masktpl_new)
                    minval, maxval, minloc, maxloc = cv.minMaxLoc(result)
                    if maxval > 0.9:
                        loc = maxloc                    
                        goc = angle
                        res = result.copy()
                top_left = (loc[0]+ x - 2, loc[1] + y - 2)
                ans_new.append([res, goc, top_left])
                print('goc topleft val', goc, top_left)
            ans = list(ans_new)
            print('----')
                
    return ans

In [37]:
def fastTemplateMatch(refname, tplname, maxleval):
    refimg = cv.imread(refname,0)
    tplimg = cv.imread(tplname,0)
    W, H = tplimg.shape[::-1]
    ## Call fastTemplateMatchInPyramid()
    start_time = time.time()
    ans = fastTemplateMatchPyramid(refimg, tplimg, maxleval)
    print("--- %s seconds ---" % (time.time() - start_time))
#     plt.imshow(ans,cmap='gray')
#     plt.show()
    for result, goc, topleft in ans:
        print('goc ', goc)
        minval, maxval, minloc, maxloc = cv.minMaxLoc(result)
        threshold = (maxval-1e6)/1.5
        loc = np.where(result >= threshold)
        tpl_new = rotation(tplimg, goc)
        Wt, Ht = tpl_new.shape[::-1]
        for pt in zip(*loc[::-1]):   
            recangleX(refimg, W, H, Wt, Ht, goc, (pt[0]+topleft[0], pt[1]+topleft[1]))
    plt.subplot()
    plt.imshow(refimg,cmap = 'gray')
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    plt.show()
    return 

In [38]:
refname = "messi_4.jpg"
tplname = "messi_face_template.jpg"
# refname = "BAT0001.PNG"
# tplname = "BAT.PNG"
# refname = "./Images/circlips_15.tif"
# tplname = "./Images/circlips01.tif"

maxleval = 5
nob = 4

fastTemplateMatch(refname, tplname, maxleval)

Level 0
18 25
53
Level 1
36 49
27
0.0 (9, 4)
-53 54
goc topleft val 52.3 (19, 6)
36 49
27
5.3 (41, 4)
0 107
goc topleft val 57.6 (80, 6)
36 49
27
10.6 (7, 3)
53 160
goc topleft val 62.9 (12, 4)
36 49
27
15.9 (6, 3)
106 213
goc topleft val 68.2 (11, 6)
36 49
27
21.2 (39, 3)
159 266
goc topleft val 73.5 (79, 7)
36 49
27
26.5 (5, 2)
212 319
goc topleft val 78.8 (11, 3)
36 49
27
31.8 (38, 2)
265 372
goc topleft val 84.1 (77, 2)
36 49
27
37.1 (38, 2)
318 425
goc topleft val 89.4 (77, 2)
36 49
27
42.4 (22, 7)
371 478
goc topleft val 94.7 (42, 15)
36 49
27
47.7 (22, 6)
424 531
goc topleft val 100.1 (42, 10)
36 49
27
53.0 (55, 5)
477 584
goc topleft val 105.3 (111, 8)
36 49
27
58.3 (55, 6)
530 637
goc topleft val 110.5 (111, 10)
36 49
27
63.6 (55, 10)
583 690
goc topleft val 102.4 (111, 18)
36 49
27
68.9 (96, 0)
636 743


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-71670poj\opencv\modules\imgproc\src\templmatch.cpp:768: error: (-215:Assertion failed) _img.size().height >= _templ.size().height && _img.size().width >= _templ.size().width in function 'cv::matchTemplateMask'


In [15]:
pwd

'C:\\Users\\DSQT\\JupyterNotebook'